<a href="https://colab.research.google.com/github/Madhuram2901/AcWOC-MovieMusic/blob/main/MovMusic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install streamlit
!pip install pyngrok

In [ ]:
%%writefile movie_music_recommender.py
import streamlit as st
import pandas as pd
import json
from difflib import get_close_matches
from typing import List, Dict, Union

st.set_page_config(
    page_title="Movie-to-Music Recommender",
    page_icon="🎬",
    layout="wide",
    initial_sidebar_state="expanded"
)

st.markdown("""
    <style>
        @import url('https://fonts.googleapis.com/css2?family=Inter:wght@400;600;700&display=swap');
        
        .stApp {
            background: #f0f2f6;
        }
        
        .main-header {
            font-family: 'Inter', sans-serif;
            font-size: 2.2rem;
            font-weight: 700;
            background: linear-gradient(90deg, #1E3A8A 0%, #3B82F6 100%);
            -webkit-background-clip: text;
            -webkit-text-fill-color: transparent;
            text-align: center;
            padding: 1.5rem 0;
            margin-bottom: 2rem;
        }
        
        .subheader {
            font-family: 'Inter', sans-serif;
            font-size: 1.5rem;
            font-weight: 600;
            color: #1E3A8A;
            margin: 1rem 0;
        }
        
        .section-container {
            background: white;
            border-radius: 12px;
            padding: 1.5rem;
            box-shadow: 0 4px 6px -1px rgba(0, 0, 0, 0.1), 0 2px 4px -1px rgba(0, 0, 0, 0.06);
            margin-bottom: 1.5rem;
        }
        
        .section-title {
            font-family: 'Inter', sans-serif;
            font-size: 1.2rem;
            font-weight: 600;
            color: #1E3A8A;
            margin-bottom: 1rem;
            display: flex;
            align-items: center;
            gap: 0.5rem;
        }
        
        .movie-card {
            background: linear-gradient(to right, #EFF6FF, #FFFFFF);
            border-radius: 12px;
            padding: 1.5rem;
            margin-bottom: 1rem;
            border-left: 4px solid #3B82F6;
        }
        
        .movie-title {
            font-family: 'Inter', sans-serif;
            font-size: 1.4rem;
            font-weight: 600;
            color: #1E3A8A;
            margin-bottom: 0.5rem;
        }
        
        .movie-overview {
            color: #4B5563;
            font-size: 1rem;
            line-height: 1.6;
            margin: 1rem 0;
        }
        
        .genre-pill {
            display: inline-block;
            background: #EFF6FF;
            color: #2563EB;
            padding: 0.25rem 0.75rem;
            border-radius: 9999px;
            font-size: 0.875rem;
            margin: 0.25rem;
            border: 1px solid #BFDBFE;
        }
        
        .track-card {
            background: white;
            border-radius: 10px;
            padding: 1.25rem;
            margin-bottom: 1rem;
            border: 1px solid #E5E7EB;
            transition: all 0.2s ease;
        }
        
        .track-card:hover {
            box-shadow: 0 4px 6px -1px rgba(0, 0, 0, 0.1), 0 2px 4px -1px rgba(0, 0, 0, 0.06);
            transform: translateY(-2px);
        }
        
        .track-title {
            font-family: 'Inter', sans-serif;
            font-size: 1.1rem;
            font-weight: 600;
            color: #1E3A8A;
            margin-bottom: 0.5rem;
        }
        
        .track-genre {
            color: #6B7280;
            font-size: 0.9rem;
            margin-bottom: 0.5rem;
        }
        
        .rating-container {
            background: #F3F4F6;
            border-radius: 8px;
            padding: 1rem;
            margin: 0.5rem 0;
        }
        
        .saved-track-card {
            background: white;
            border-radius: 10px;
            padding: 1rem;
            margin-bottom: 0.75rem;
            border: 1px solid #E5E7EB;
            transition: all 0.2s ease;
        }
        
        .saved-track-card:hover {
            background: #F8FAFC;
        }
        
        .input-container {
            background: white;
            border-radius: 8px;
            padding: 1rem;
            margin-bottom: 1rem;
            border: 1px solid #E5E7EB;
        }
        
        .stButton button {
            background: #2563EB;
            color: white;
            border-radius: 6px;
            padding: 0.5rem 1rem;
            font-weight: 500;
            border: none;
            transition: all 0.2s ease;
        }
        
        .stButton button:hover {
            background: #1D4ED8;
            box-shadow: 0 4px 6px -1px rgba(0, 0, 0, 0.1), 0 2px 4px -1px rgba(0, 0, 0, 0.06);
        }
        
        .stSelectbox > div > div {
            background: white;
            border-radius: 6px;
            border: 1px solid #E5E7EB;
        }
        
        div[data-testid="stToolbar"] {
            display: none;
        }
        
        footer {
            display: none;
        }
        
        .stAlert {
            background: white;
            border-radius: 10px;
            border: 1px solid #E5E7EB;
        }
        
        .metric-container {
            background: #F8FAFC;
            border-radius: 8px;
            padding: 1rem;
            margin: 0.5rem 0;
            border: 1px solid #E5E7EB;
            text-align: center;
        }
        
        .metric-value {
            font-size: 1.5rem;
            font-weight: 600;
            color: #1E3A8A;
        }
        
        .metric-label {
            font-size: 0.875rem;
            color: #6B7280;
            margin-top: 0.25rem;
        }
    </style>
""", unsafe_allow_html=True)

class GenreRecommendationSystem:
    def __init__(self, movies_path: str, music_path: str):
        self.movies_df = self._load_movie_data(movies_path)
        self.music_genres_df = self._load_music_data(music_path)
        self.genre_mapping = self._create_genre_mapping()

    def _load_movie_data(self, path: str) -> pd.DataFrame:
        try:
            movies_df = pd.read_csv(path, usecols=['id', 'genres', 'title', 'overview', 'vote_average'])
            movies_df['genres'] = movies_df['genres'].apply(lambda x: [genre['name'] for genre in json.loads(x)])
            return movies_df
        except Exception as e:
            raise ValueError(f"Error loading movie data: {e}")

    def _load_music_data(self, path: str) -> pd.DataFrame:
        try:
            music_df = pd.read_csv(path)
            required_columns = ['genres', 'track_names']
            missing_columns = [col for col in required_columns if col not in music_df.columns]
            if missing_columns:
                raise ValueError(f"Missing columns in music data: {missing_columns}")
            return music_df
        except Exception as e:
            raise ValueError(f"Error loading music data: {e}")

    def _create_genre_mapping(self) -> Dict[str, List[str]]:
        return {
            'Action': ['electronic', 'rock', 'epic', 'bass music', 'drum and bass'],
            'Adventure': ['world', 'cinematic', 'orchestral', 'folk'],
            'Fantasy': ['21st century classical', 'orchestral', 'cinematic', 'world'],
            'Science Fiction': ['electronic', 'ambient', 'abstract', 'abstract beats'],
            'Crime': ['dark ambient', 'abstract hip hop', 'trip hop'],
            'Drama': ['classical', 'acoustic', 'piano', 'ambient'],
            'Comedy': ['a cappella', 'pop', 'quirky', 'fun'],
            'Romance': ['acoustic', 'piano', 'vocal'],
            'Horror': ['dark ambient', 'abstract', 'atmospheric'],
            'Thriller': ['dark ambient', 'electronic', 'atmospheric'],
            'Animation': ['8-bit', 'fun', 'quirky', 'orchestral'],
            'Family': ['acoustic', 'pop', 'folk', 'fun']
        }

    def find_movie(self, movie_title: str) -> Union[None, pd.Series]:
        matches = self.movies_df[self.movies_df['title'].str.contains(movie_title, case=False, na=False)]
        return matches.iloc[0] if not matches.empty else None

    def get_related_music_genres(self, movie_genres: List[str]) -> List[str]:
        related_music_genres = set()
        for genre in movie_genres:
            if genre in self.genre_mapping:
                related_music_genres.update(self.genre_mapping[genre])
        return list(related_music_genres)

    def recommend_music_based_on_movie(self, movie_title: str, num_recommendations: int = 5, genre_filter: List[str] = None) -> Union[List[Dict[str, str]], str]:
        movie = self.find_movie(movie_title)
        if movie is None:
            return f"Movie not found. Did you mean one of these: {', '.join(get_close_matches(movie_title, self.movies_df['title'].tolist(), n=3))}?"

        movie_genres = movie['genres']
        related_music_genres = self.get_related_music_genres(movie_genres)

        if not related_music_genres:
            return "No matching music genres found for this movie's genres."

        if genre_filter:
            related_music_genres = [genre for genre in related_music_genres if genre in genre_filter]

        matching_music = self.music_genres_df[
            self.music_genres_df['genres'].str.lower().isin([g.lower() for g in related_music_genres])
        ]

        if matching_music.empty:
            return "No music recommendations found for the related genres."

        recommendations = matching_music.sample(n=min(num_recommendations, len(matching_music)))
        return recommendations[['genres', 'track_names']].to_dict('records')

@st.cache_data
def load_recommender(movies_path: str, music_path: str) -> GenreRecommendationSystem:
    return GenreRecommendationSystem(movies_path=movies_path, music_path=music_path)

def save_tracks_to_file(file_path: str, tracks: List[Dict[str, Union[str, int]]]):
    with open(file_path, 'w') as file:
        for track in tracks:
            file.write(json.dumps(track) + '\n')

def load_tracks_from_file(file_path: str) -> List[Dict[str, Union[str, int]]]:
    try:
        with open(file_path, 'r') as file:
            return [json.loads(line) for line in file]
    except FileNotFoundError:
        return []

def render_movie_details(movie):
    st.markdown('<div class="movie-card">', unsafe_allow_html=True)
    st.markdown(f'<div class="movie-title">🎬 {movie["title"]}</div>', unsafe_allow_html=True)
    
    col1, col2, col3 = st.columns(3)
    with col1:
        st.markdown('<div class="metric-container">', unsafe_allow_html=True)
        st.markdown(f'<div class="metric-value">⭐ {movie["vote_average"]}/10</div>', unsafe_allow_html=True)
        st.markdown('<div class="metric-label">Rating</div>', unsafe_allow_html=True)
        st.markdown('</div>', unsafe_allow_html=True)
    
    st.markdown('<div class="movie-overview">', unsafe_allow_html=True)
    st.write(movie['overview'])
    st.markdown('</div>', unsafe_allow_html=True)
    
    st.markdown('<div style="margin-top: 1rem;">', unsafe_allow_html=True)
    for genre in movie['genres']:
        st.markdown(f'<span class="genre-pill">{genre}</span>', unsafe_allow_html=True)
    st.markdown('</div>', unsafe_allow_html=True)
    
    st.markdown('</div>', unsafe_allow_html=True)

def render_track_recommendation(rec, index, movie_title, saved_tracks):
    st.markdown('<div class="track-card">', unsafe_allow_html=True)
    st.markdown(f'<div class="track-title">🎵 {rec["track_names"]}</div>', unsafe_allow_html=True)
    st.markdown(f'<div class="track-genre">Genre: {rec["genres"]}</div>', unsafe_allow_html=True)
    
    col1, col2 = st.columns([3, 1])
    with col1:
        st.markdown('<div class="rating-container">', unsafe_allow_html=True)
        rating = st.slider(
            "Rate this track",
            1, 5, 3,
            key=f"rating_{index}_{movie_title}",
            help="1 = Poor, 5 = Excellent"
        )
        st.markdown('</div>', unsafe_allow_html=True)
    
    with col2:
        if st.button("💾 Save", key=f"save_{index}_{movie_title}"):
            track_data = {
                'movie': movie_title,
                'track': rec['track_names'],
                'genre': rec['genres'],
                'rating': rating
            }
            saved_tracks.append(track_data)
            save_tracks_to_file("saved_tracks.txt", saved_tracks)
            st.success("✅ Track saved!")
    
    st.markdown('</div>', unsafe_allow_html=True)

def main():
    st.markdown('<h1 class="main-header">Movie-to-Music Recommender</h1>', unsafe_allow_html=True)
    
    container = st.container()
    with container:
        col1, col2 = st.columns([2, 1])
        
        with col2:
            st.markdown('<div class="section-container">', unsafe_allow_html=True)
            st.markdown('<div class="section-title">🛠️ Configuration</div>', unsafe_allow_html=True)
            
            movies_path = st.text_input(
                "Movies Dataset Path",
                "tmdb_5000_movies.csv",
                help="Path to your movies dataset CSV file"
            )
            
            music_path = st.text_input(
                "Music Dataset Path",
                "extended_data_by_genres.csv",
                help="Path to your music dataset CSV file"
            )
            st.markdown('</div>', unsafe_allow_html=True)
            
            st.markdown('<div class="section-container">', unsafe_allow_html=True)
            st.markdown('<div class="section-title">🎵 Music Preferences</div>', unsafe_allow_html=True)
            genre_filter = st.multiselect(
                "Filter by Music Genres",
                options=[
                    'electronic', 'rock', 'epic', 'bass music', 'drum and bass',
                    'world', 'cinematic', 'orchestral', 'folk', '21st century classical',
                    'ambient', 'abstract', 'abstract beats', 'dark ambient',
                    'abstract hip hop', 'trip hop', 'classical', 'acoustic', 'piano',
                    'a cappella', 'pop', 'quirky', 'fun', 'vocal', 'atmospheric', '8-bit'
                ],
                help="Select specific music genres to filter recommendations"
            )
            st.markdown('</div>', unsafe_allow_html=True)

        with col1:
            saved_tracks_file = "saved_tracks.txt"

            if not movies_path or not music_path:
                st.error("⚠️ Please provide both movies and music dataset paths.")
                return

            try:
                recommender = load_recommender(movies_path, music_path)
            except ValueError as e:
                st.error(f"⚠️ {e}")
                return

            saved_tracks = load_tracks_from_file(saved_tracks_file)
            if 'current_recommendations' not in st.session_state:
                st.session_state['current_recommendations'] = []

            st.markdown('<div class="section-container">', unsafe_allow_html=True)
            st.markdown('<div class="section-title">🔍 Find Your Movie</div>', unsafe_allow_html=True)
            movie_title = st.text_input(
                "",
                placeholder="Enter a movie title (e.g., The Dark Knight, Avatar, Inception...)",
                help="Type the name of a movie to get music recommendations"
            )
            st.markdown('</div>', unsafe_allow_html=True)

            if movie_title:
                recommendations = recommender.recommend_music_based_on_movie(
                    movie_title,
                    genre_filter=genre_filter
                )

                if isinstance(recommendations, str):
                    st.error(f"🚫 {recommendations}")
                else:
                    movie = recommender.find_movie(movie_title)
                    render_movie_details(movie)

                    if not st.session_state['current_recommendations']:
                        st.session_state['current_recommendations'] = recommendations

                    st.markdown('<div class="section-container">', unsafe_allow_html=True)
                    st.markdown('<div class="section-title">🎵 Recommended Tracks</div>', unsafe_allow_html=True)
                    
                    for i, rec in enumerate(st.session_state['current_recommendations'], 1):
                        render_track_recommendation(rec, i, movie_title, saved_tracks)
                    
                    st.markdown('</div>', unsafe_allow_html=True)

    st.markdown('<div class="section-container">', unsafe_allow_html=True)
    st.markdown('<div class="section-title">📚 Your Music Collection</div>', unsafe_allow_html=True)
    
    if saved_tracks:
        for track in saved_tracks:
            st.markdown('<div class="saved-track-card">', unsafe_allow_html=True)
            cols = st.columns([2, 2, 1, 1])
            with cols[0]:
                st.markdown(f"<strong>🎬 {track['movie']}</strong>", unsafe_allow_html=True)
            with cols[1]:
                st.markdown(f"<strong>🎵 {track['track']}</strong>", unsafe_allow_html=True)
            with cols[2]:
                st.markdown(f"<span class='genre-pill'>{track['genre']}</span>", unsafe_allow_html=True)
            with cols[3]:
                st.markdown(f"<div style='text-align: right;'>{'⭐' * track['rating']}</div>", unsafe_allow_html=True)
            st.markdown('</div>', unsafe_allow_html=True)
    else:
        st.markdown("""
            <div style="text-align: center; padding: 2rem; color: #6B7280;">
                <div style="font-size: 2rem; margin-bottom: 1rem;">📝</div>
                <div style="font-weight: 500; margin-bottom: 0.5rem;">No saved tracks yet</div>
                <div style="font-size: 0.875rem;">Start by searching for a movie and saving some tracks you like!</div>
            </div>
        """, unsafe_allow_html=True)
    
    st.markdown('</div>', unsafe_allow_html=True)

if __name__ == "__main__":
    main()

Writing movie_music_recommender.py


In [ ]:
from pyngrok import ngrok
import os

# Get your authtoken from https://dashboard.ngrok.com
# and replace 'YOUR_AUTHTOKEN' with your actual token
ngrok.set_auth_token("YOUR_AUTHTOKEN")

# Terminate existing ngrok processes
os.system("killall ngrok")

public_url = ngrok.connect(addr='8501')

os.system('streamlit run movie_music_recommender.py &')

public_url